In [12]:
import numpy as np
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.ppo import MlpPolicy
import torch

from imitation.algorithms.adversarial.gail import GAIL
from imitation.data import rollout
from imitation.data.wrappers import RolloutInfoWrapper
from imitation.policies.serialize import load_policy
from imitation.rewards.reward_nets import BasicRewardNet
from imitation.util.networks import RunningNorm
from imitation.util.util import make_vec_env

SEED = 42

In [2]:

env = make_vec_env(
    "seals:seals/CartPole-v0",
    rng=np.random.default_rng(SEED),
    n_envs=8,
    post_wrappers=[lambda env, _: RolloutInfoWrapper(env)],  # to compute rollouts
)
expert = load_policy(
    "ppo-huggingface",
    organization="HumanCompatibleAI",
    env_name="seals-CartPole-v0",
    venv=env,
)


In [3]:

rollouts = rollout.rollout(
    expert,
    env,
    rollout.make_sample_until(min_timesteps=None, min_episodes=60),
    rng=np.random.default_rng(SEED),
)


In [4]:

learner = PPO(
    env=env,
    policy=MlpPolicy,
    batch_size=64,
    ent_coef=0.0,
    learning_rate=0.0004,
    gamma=0.95,
    n_epochs=5,
    seed=SEED,
)
reward_net = BasicRewardNet(
    observation_space=env.observation_space,
    action_space=env.action_space,
    normalize_input_layer=RunningNorm,
)
gail_trainer = GAIL(
    demonstrations=rollouts,
    demo_batch_size=1024,
    gen_replay_buffer_capacity=512,
    n_disc_updates_per_round=8,
    venv=env,
    gen_algo=learner,
    reward_net=reward_net,
)


In [5]:

# evaluate the learner before training
env.seed(SEED)
learner_rewards_before_training, _ = evaluate_policy(
    learner, env, 100, return_episode_rewards=True,
)


In [7]:

# train the learner and evaluate again
gail_trainer.train(800000)  # Train for 800_000 steps to match expert.
env.seed(SEED)


round:   0%|          | 0/48 [00:00<?, ?it/s]

----------------------------------------------------
| raw/                               |             |
|    gen/rollout/ep_len_mean         | 500         |
|    gen/rollout/ep_rew_mean         | 35.3        |
|    gen/rollout/ep_rew_wrapped_mean | 270         |
|    gen/time/fps                    | 5854        |
|    gen/time/iterations             | 1           |
|    gen/time/time_elapsed           | 2           |
|    gen/time/total_timesteps        | 32768       |
|    gen/train/approx_kl             | 0.006985949 |
|    gen/train/clip_fraction         | 0.0338      |
|    gen/train/clip_range            | 0.2         |
|    gen/train/entropy_loss          | -0.687      |
|    gen/train/explained_variance    | 0.0555256   |
|    gen/train/learning_rate         | 0.0004      |
|    gen/train/loss                  | 0.0315      |
|    gen/train/n_updates             | 5           |
|    gen/train/policy_gradient_loss  | -0.00158    |
|    gen/train/value_loss            | 4.7    

round:   2%|▏         | 1/48 [00:05<04:40,  5.97s/it]

----------------------------------------------------
| raw/                               |             |
|    gen/rollout/ep_len_mean         | 500         |
|    gen/rollout/ep_rew_mean         | 33.3        |
|    gen/rollout/ep_rew_wrapped_mean | 284         |
|    gen/time/fps                    | 5796        |
|    gen/time/iterations             | 1           |
|    gen/time/time_elapsed           | 2           |
|    gen/time/total_timesteps        | 49152       |
|    gen/train/approx_kl             | 0.008539271 |
|    gen/train/clip_fraction         | 0.0651      |
|    gen/train/clip_range            | 0.2         |
|    gen/train/entropy_loss          | -0.677      |
|    gen/train/explained_variance    | 0.7282779   |
|    gen/train/learning_rate         | 0.0004      |
|    gen/train/loss                  | 0.0787      |
|    gen/train/n_updates             | 10          |
|    gen/train/policy_gradient_loss  | -0.00236    |
|    gen/train/value_loss            | 0.26   

round:   4%|▍         | 2/48 [00:12<04:38,  6.05s/it]

...

----------------------------------------------------
| raw/                               |             |
|    gen/rollout/ep_len_mean         | 500         |
|    gen/rollout/ep_rew_mean         | 500         |
|    gen/rollout/ep_rew_wrapped_mean | 414         |
|    gen/time/fps                    | 6280        |
|    gen/time/iterations             | 1           |
|    gen/time/time_elapsed           | 2           |
|    gen/time/total_timesteps        | 786432      |
|    gen/train/approx_kl             | 0.009355076 |
|    gen/train/clip_fraction         | 0.106       |
|    gen/train/clip_range            | 0.2         |
|    gen/train/entropy_loss          | -0.295      |
|    gen/train/explained_variance    | 0.9822101   |
|    gen/train/learning_rate         | 0.0004      |
|    gen/train/loss                  | 0.00842     |
|    gen/train/n_updates             | 235         |
|    gen/train/policy_gradient_loss  | -0.00484    |
|    gen/train/value_loss            | 0.0067 

round:  98%|█████████▊| 47/48 [04:24<00:05,  5.59s/it]

-----------------------------------------------------
| raw/                               |              |
|    gen/rollout/ep_len_mean         | 500          |
|    gen/rollout/ep_rew_mean         | 500          |
|    gen/rollout/ep_rew_wrapped_mean | 408          |
|    gen/time/fps                    | 6258         |
|    gen/time/iterations             | 1            |
|    gen/time/time_elapsed           | 2            |
|    gen/time/total_timesteps        | 802816       |
|    gen/train/approx_kl             | 0.0058190124 |
|    gen/train/clip_fraction         | 0.0626       |
|    gen/train/clip_range            | 0.2          |
|    gen/train/entropy_loss          | -0.278       |
|    gen/train/explained_variance    | 0.9946737    |
|    gen/train/learning_rate         | 0.0004       |
|    gen/train/loss                  | -0.00358     |
|    gen/train/n_updates             | 240          |
|    gen/train/policy_gradient_loss  | -0.00311     |
|    gen/train/value_loss   

round: 100%|██████████| 48/48 [04:30<00:00,  5.63s/it]


[42, 43, 44, 45, 46, 47, 48, 49]

In [8]:

learner_rewards_after_training, _ = evaluate_policy(
    learner, env, 100, return_episode_rewards=True,
)


In [9]:

print("mean reward after training:", np.mean(learner_rewards_after_training))
print("mean reward before training:", np.mean(learner_rewards_before_training))

mean reward after training: 500.0
mean reward before training: 102.6


In [10]:
learner.save("models/gail_cartpole_learner")

In [31]:
expert = PPO.load("models/gail_cartpole_learner", env=env)

In [35]:
env  = gym.make("seals:seals/CartPole-v0", render_mode='human')
obs, _ = env.reset()
env.render()
done = False
truncated = False   
g_reward = 0
for _ in range(10):
    while not done and not truncated:
        # Predict action
        action, _states = expert.predict(obs, deterministic=True)

        # Step in environment
        obs, reward, done, truncated, info = env.step(action)
        g_reward += reward
        # Optional: render is handled automatically with render_mode="human"
        
        print(f"Reward: {g_reward}")
env.close()

Reward: 1.0
Reward: 2.0
Reward: 3.0
Reward: 4.0
Reward: 5.0
Reward: 6.0
Reward: 7.0
Reward: 8.0
Reward: 9.0
Reward: 10.0
Reward: 11.0
Reward: 12.0
Reward: 13.0
Reward: 14.0
Reward: 15.0
Reward: 16.0
Reward: 17.0
Reward: 18.0
Reward: 19.0
Reward: 20.0
Reward: 21.0
Reward: 22.0
Reward: 23.0
Reward: 24.0
Reward: 25.0
Reward: 26.0
Reward: 27.0
Reward: 28.0
Reward: 29.0
Reward: 30.0
Reward: 31.0
Reward: 32.0
Reward: 33.0
Reward: 34.0
Reward: 35.0
Reward: 36.0
Reward: 37.0
Reward: 38.0
Reward: 39.0
Reward: 40.0
Reward: 41.0
Reward: 42.0
Reward: 43.0
Reward: 44.0
Reward: 45.0
Reward: 46.0
Reward: 47.0
Reward: 48.0
Reward: 49.0
Reward: 50.0
Reward: 51.0
Reward: 52.0
Reward: 53.0
Reward: 54.0
Reward: 55.0
Reward: 56.0
Reward: 57.0
Reward: 58.0
Reward: 59.0
Reward: 60.0
Reward: 61.0
Reward: 62.0
Reward: 63.0
Reward: 64.0
Reward: 65.0
Reward: 66.0
Reward: 67.0
Reward: 68.0
Reward: 69.0
Reward: 70.0
Reward: 71.0
Reward: 72.0
Reward: 73.0
Reward: 74.0
Reward: 75.0
Reward: 76.0
Reward: 77.0
Reward: 